# step1-offline-process

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from zhipuai import ZhipuAI
from langchain_community.embeddings import ZhipuAIEmbeddings
# 1. 加载文档
loader = TextLoader("knowledge_base.txt")
documents = loader.load()

# 2. 文本切分
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 每个文本块的大小
    chunk_overlap=50,  # 文本块之间的重叠部分
)
splits = text_splitter.split_documents(documents)

# 3. 向量化并存储
embeddings = ZhipuAIEmbeddings(
    api_key="", # 在 bigmodel.cn 控制台获取
    model="embedding-2" # 智谱的文本向量模型
)

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="./chroma_db",  # 持久化存储路径
)

print(f"成功将 {len(splits)} 个文本块存入向量数据库")


成功将 1 个文本块存入向量数据库


In [3]:
splits

[Document(metadata={'source': 'knowledge_base.txt'}, page_content='RAG 的的本质是：增强 System Prompt 的上下文，从而生成更符合实际要求的回复。')]

# step2-online-rag

In [22]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage

# 1. 加载已有的向量数据库
embeddings = ZhipuAIEmbeddings(
    api_key="", # 在 bigmodel.cn 控制台获取
    model="embedding-2" # 智谱的文本向量模型
)
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

# 2. 用户提问
query = "什么是RAG？"

# 3. 检索相关文档（返回最相关的 3 个）
docs = vectorstore.similarity_search(query, k=3)

# 4. 将检索到的文档内容拼接成上下文
context = "\n\n".join([doc.page_content for doc in docs])

# 5. 构建 Prompt 模板
prompt_template = """
你是一个专业的问答助手。请根据以下参考文档回答用户的问题。
如果参考文档中没有相关信息，请诚实地说不知道，不要编造答案。

参考文档：
{context}

用户问题：{question}

回答：
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
)

# 6. 创建 LLM 并生成回答

llm = ChatOpenAI(
    openai_api_key="",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4",
    model_name="glm-4",
    temperature=0
)

final_prompt = prompt.format(context=context, question=query)

print(f"最终的 Prompt 内容：{final_prompt}")

messages = [HumanMessage(content=final_prompt)]
response = llm.invoke(messages)

# 7. 输出结果
print(f"问题: {query}")
print(f"回答: {response.content}")
print(f"\n参考文档数量: {len(docs)}")


最终的 Prompt 内容：
你是一个专业的问答助手。请根据以下参考文档回答用户的问题。
如果参考文档中没有相关信息，请诚实地说不知道，不要编造答案。

参考文档：
RAG 的的本质是：增强 System Prompt 的上下文，从而生成更符合实际要求的回复。

用户问题：什么是RAG？

回答：

问题: 什么是RAG？
回答: RAG的本质是增强System Prompt的上下文，从而生成更符合实际要求的回复。简单来说，RAG是一种技术，它通过提升系统提示的上下文信息，使得生成的回答更加准确和符合用户的需求。

参考文档数量: 1
